In [10]:
import sys
sys.path.append("../") 
from libs.cna_utils import *
np.random.seed(0)

# CASE/CONTROL

## Single Cell

In [2]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/case_control/sc_meta.csv')
print(meta.shape)

(130521, 41)


In [3]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/case_control/sc_harmony.csv')
print(harmony.shape)

(130521, 20)


In [4]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/case_control/sc_umap.csv')
print(umap.shape)

(130521, 2)


In [5]:
# Start by making anndata object
d = mad.MultiAnnData(X=harmony, obs=meta, sampleid="sample")
#d.obs_to_sample(['Type', 'Site',
#                 'Responder_Status', 'Sex',
#                 'Age', 'Race',
#                'Ethnicity', 'ISN',
#               'Activity', "Chronicity",
#                 'processing_batch'])
d.obs_to_sample(['Case.Control'])
d.samplem.head()

/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


['cell' 'sample' 'final_annotation' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies


,Case.Control
sample,
AMPSLEkid_cells_0373,1.0
AMPSLEkid_cells_0382,1.0
AMPSLEkid_cells_0964,1.0
AMPSLEkid_cells_1135,1.0
AMPSLEkid_cells_1154,1.0


In [6]:
umap.index = d.obs.index
d.obsm['X_umap'] = umap

In [7]:
np.random.seed(0) 
cna.pp.knn(d)

computing default knn graph


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem['Case.Control'], #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 63.71152114868164
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 41.964599609375
	20th percentile R2(t,t-1): 0.763817548751831
	taking step 3
	median kurtosis: 28.33036994934082
	20th percentile R2(t,t-1): 0.9081483483314514
	taking step 4
	median kurtosis: 19.45665168762207
	20th percentile R2(t,t-1): 0.9424880146980286
	taking step 5
	median kurtosis: 14.009087562561035
	20th percentile R2(t,t-1): 0.9607177376747131
	taking step 6
	median kurtosis: 10.741266250610352
	20th percentile R2(t,t-1): 0.9730812907218933
	taking step 7
	median kurtosis: 8.7838773727417
	20th percentile R2(t,t-1): 0.9826822876930237
stopping after 7 steps
covariate-adjusted NAM not found; computing and saving


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing SVD
performing association test


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_association.py:74: UserWarning: global association p-value attained minimal possible value. Consider increasing Nnull
  warnings.warn('global association p-value attained minimal possible value. '+\


computing neighborhood-level FDRs
18
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 9.999000099990002e-05 , 18 PCs used
total r^2 between top 18 NAM PCs and outcome is 0.55


In [9]:
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/case_control/sc_ncorr.csv", 
               res.ncorrs, delimiter=",")
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/case_control/sc_fdrs.csv", 
               res.fdrs, delimiter=",")

## Single Nuclear

In [10]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/case_control/sn_meta.csv')
print(meta.shape)

(42726, 35)


In [11]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/case_control/sn_harmony.csv')
print(harmony.shape)

(42726, 20)


In [12]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/case_control/sn_umap.csv')
print(umap.shape)

(42726, 2)


In [13]:
# Start by making anndata object
d = mad.MultiAnnData(X=harmony, obs=meta, sampleid="sample")
#d.obs_to_sample(['Type', 'Site',
#                 'Responder_Status', 'Sex',
#                 'Age', 'Race',
#                'Ethnicity', 'ISN',
#               'Activity', "Chronicity",
#                 'processing_batch'])
d.obs_to_sample(['Case.Control'])
d.samplem.head()

['cell' 'sample' 'final_annotation' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


,Case.Control
sample,
AMPSLEkid_nuc_0147,1.0
AMPSLEkid_nuc_0366,1.0
AMPSLEkid_nuc_0849,1.0
AMPSLEkid_nuc_0880,1.0
AMPSLEkid_nuc_0883,1.0


In [14]:
umap.index = d.obs.index
d.obsm['X_umap'] = umap

In [15]:
np.random.seed(0) 
cna.pp.knn(d)

computing default knn graph


In [16]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem['Case.Control'], #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 15.079200744628906
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 10.210453987121582
	20th percentile R2(t,t-1): 0.7660784125328064
	taking step 3
	median kurtosis: 6.767342567443848
	20th percentile R2(t,t-1): 0.9072574973106384
	taking step 4
	median kurtosis: 4.982381582260132
	20th percentile R2(t,t-1): 0.9467245459556579
stopping after 4 steps
covariate-adjusted NAM not found; computing and saving
computing SVD
performing association test


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_association.py:74: UserWarning: global association p-value attained minimal possible value. Consider increasing Nnull
  warnings.warn('global association p-value attained minimal possible value. '+\


computing neighborhood-level FDRs
7
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 9.999000099990002e-05 , 7 PCs used
total r^2 between top 7 NAM PCs and outcome is 0.82


In [17]:
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/case_control/sn_ncorr.csv", 
               res.ncorrs, delimiter=",")
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/case_control/sn_fdrs.csv", 
               res.fdrs, delimiter=",")

# AGE

## Single Cell

In [2]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/age_sex_response_race/sc_meta_age.csv')
print(meta.shape)

(116401, 39)


In [3]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/age_sex_response_race/sc_harmony_age.csv')
print(harmony.shape)

(116401, 20)


In [4]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/age_sex_response_race/sc_umap_age.csv')
print(umap.shape)

(116401, 2)


In [5]:
# Start by making anndata object
d = mad.MultiAnnData(X=harmony, obs=meta, sampleid="sample")
#d.obs_to_sample(['Type', 'Site',
#                 'Responder_Status', 'Sex',
#                 'Age', 'Race',
#                'Ethnicity', 'ISN',
#               'Activity', "Chronicity",
#                 'processing_batch'])
d.obs_to_sample(['Sex', 'Age', 'Final_Chronicity', 'Final_Activity', 'First_biop', 'Pred_use', 'Responder_Status', 'Race_[A]',
       'Race_[A][B]', 'Race_[B]', 'Race_[B][AI]', 'Race_[B][W]', 'Race_[U]',
       'Race_[W]', 'Final_ISN_[III]', 'Final_ISN_[III][V]', 'Final_ISN_[IV]',
       'Final_ISN_[IV][V]', 'Final_ISN_[V]', 'Final_Site_Cincinnati',
       'Final_Site_Einstein', 'Final_Site_JHU', 'Final_Site_Michigan',
       'Final_Site_MUSC', 'Final_Site_Northwell', 'Final_Site_NYU',
       'Final_Site_Rochester', 'Final_Site_Texas Tech', 'Final_Site_UCLA',
       'Final_Site_UCSD', 'Final_Site_UCSF'])
d.samplem.head()

/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


['cell' 'sample' 'cluster_names' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies


,Sex,Age,Final_Chronicity,Final_Activity,First_biop,Pred_use,Responder_Status,Race_[A],Race_[A][B],Race_[B],...,Final_Site_JHU,Final_Site_Michigan,Final_Site_MUSC,Final_Site_Northwell,Final_Site_NYU,Final_Site_Rochester,Final_Site_Texas Tech,Final_Site_UCLA,Final_Site_UCSD,Final_Site_UCSF
sample,,,,,,,,,,,,,,,,,,,,,
AMPSLEkid_cells_0373,1.0,-0.316264,5.0,3.0,0.0,1.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
AMPSLEkid_cells_0382,1.0,1.904141,3.0,8.0,1.0,1.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
AMPSLEkid_cells_0964,1.0,-1.255666,3.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
AMPSLEkid_cells_1135,1.0,-1.341067,3.0,7.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_1154,1.0,0.708538,3.0,7.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [9]:
umap.index = d.obs.index
d.obsm['X_umap'] = umap

In [10]:
np.random.seed(0) 
cna.pp.knn(d)

computing default knn graph


In [11]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Age, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 53.41489028930664
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 35.2132568359375
	20th percentile R2(t,t-1): 0.7658944725990295
	taking step 3
	median kurtosis: 23.83907127380371
	20th percentile R2(t,t-1): 0.9089255332946777
	taking step 4
	median kurtosis: 16.483577728271484
	20th percentile R2(t,t-1): 0.9428997039794922
	taking step 5
	median kurtosis: 12.018216133117676
	20th percentile R2(t,t-1): 0.9613077640533447
	taking step 6
	median kurtosis: 9.285207748413086
	20th percentile R2(t,t-1): 0.9742846488952637
stopping after 6 steps
covariate-adjusted NAM not found; computing and saving


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing SVD
performing association test
computing neighborhood-level FDRs
19
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.026697330266973303 , 19 PCs used
total r^2 between top 19 NAM PCs and outcome is 0.22


In [13]:
sc_uni['Age'] = res.p

In [14]:
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/age_sex_response_race/sc_age_ncorr.csv", 
               res.ncorrs, delimiter=",")
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/age_sex_response_race/sc_age_fdrs.csv", 
               res.fdrs, delimiter=",")

## Single Nuclei

In [11]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/age_sex_response_race/sn_meta_age.csv')
print(meta.shape)

(36541, 9)


In [12]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/age_sex_response_race/sn_harmony_age.csv')
print(harmony.shape)

(36541, 20)


In [13]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/age_sex_response_race/sn_umap_age.csv')
print(umap.shape)

(36541, 2)


In [14]:
# Start by making anndata object
d = mad.MultiAnnData(X=harmony, obs=meta, sampleid="sample")
#d.obs_to_sample(['Type', 'Site',
#                 'Responder_Status', 'Sex',
#                 'Age', 'Race',
#                'Ethnicity', 'ISN',
#               'Activity', "Chronicity",
#                 'processing_batch'])
d.obs_to_sample(['Age'])
d.samplem.head()

['cell' 'Sex' 'sample' 'cluster_names' 'Responder.Status' 'Race' 'ISN'
 'Type']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


,Age
sample,
AMPSLEkid_cells_0147,20.0
AMPSLEkid_cells_0366,3.0
AMPSLEkid_cells_0849,14.0
AMPSLEkid_cells_0880,21.0
AMPSLEkid_cells_0883,1.0


In [15]:
umap.index = d.obs.index
d.obsm['X_umap'] = umap

In [17]:
np.random.seed(0) 
cna.pp.knn(d)

computing default knn graph


In [18]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Age, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 12.009225845336914
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 8.349165916442871
	20th percentile R2(t,t-1): 0.7731621861457825
	taking step 3
	median kurtosis: 5.703102111816406
	20th percentile R2(t,t-1): 0.9103659629821778
stopping after 3 steps
covariate-adjusted NAM not found; computing and saving
computing SVD
performing association test


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing neighborhood-level FDRs
18
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.0471952804719528 , 18 PCs used
total r^2 between top 18 NAM PCs and outcome is 0.67


In [19]:
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/age_sex_response_race/sn_age_ncorr.csv", 
               res.ncorrs, delimiter=",")
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/age_sex_response_race/sn_age_fdrs.csv", 
               res.fdrs, delimiter=",")

# SEX

## Single Cell

In [15]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/age_sex_response_race/sc_meta_sex.csv')
print(meta.shape)

(116401, 39)


In [18]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/age_sex_response_race/sc_harmony_sex.csv')
print(harmony.shape)

(116401, 20)


In [17]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/age_sex_response_race/sc_umap_sex.csv')
print(umap.shape)

(116401, 2)


In [19]:
# Start by making anndata object
d = mad.MultiAnnData(X=harmony, obs=meta, sampleid="sample")
#d.obs_to_sample(['Type', 'Site',
#                 'Responder_Status', 'Sex',
#                 'Age', 'Race',
#                'Ethnicity', 'ISN',
#               'Activity', "Chronicity",
#                 'processing_batch'])
d.obs_to_sample(['Sex', 'Age', 'Final_Chronicity', 'Final_Activity', 'First_biop', 'Pred_use', 'Responder_Status', 'Race_[A]',
       'Race_[A][B]', 'Race_[B]', 'Race_[B][AI]', 'Race_[B][W]', 'Race_[U]',
       'Race_[W]', 'Final_ISN_[III]', 'Final_ISN_[III][V]', 'Final_ISN_[IV]',
       'Final_ISN_[IV][V]', 'Final_ISN_[V]', 'Final_Site_Cincinnati',
       'Final_Site_Einstein', 'Final_Site_JHU', 'Final_Site_Michigan',
       'Final_Site_MUSC', 'Final_Site_Northwell', 'Final_Site_NYU',
       'Final_Site_Rochester', 'Final_Site_Texas Tech', 'Final_Site_UCLA',
       'Final_Site_UCSD', 'Final_Site_UCSF'])
d.samplem.head()

/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


['cell' 'sample' 'cluster_names' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies


,Sex,Age,Final_Chronicity,Final_Activity,First_biop,Pred_use,Responder_Status,Race_[A],Race_[A][B],Race_[B],...,Final_Site_JHU,Final_Site_Michigan,Final_Site_MUSC,Final_Site_Northwell,Final_Site_NYU,Final_Site_Rochester,Final_Site_Texas Tech,Final_Site_UCLA,Final_Site_UCSD,Final_Site_UCSF
sample,,,,,,,,,,,,,,,,,,,,,
AMPSLEkid_cells_0373,1.0,-0.316264,5.0,3.0,0.0,1.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
AMPSLEkid_cells_0382,1.0,1.904141,3.0,8.0,1.0,1.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
AMPSLEkid_cells_0964,1.0,-1.255666,3.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
AMPSLEkid_cells_1135,1.0,-1.341067,3.0,7.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_1154,1.0,0.708538,3.0,7.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [20]:
umap.index = d.obs.index
d.obsm['X_umap'] = umap

In [21]:
np.random.seed(0) 
cna.pp.knn(d)

computing default knn graph


In [22]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Sex, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 53.41489028930664
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 35.2132568359375
	20th percentile R2(t,t-1): 0.7658944725990295
	taking step 3
	median kurtosis: 23.83907127380371
	20th percentile R2(t,t-1): 0.9089255332946777
	taking step 4
	median kurtosis: 16.483577728271484
	20th percentile R2(t,t-1): 0.9428997039794922
	taking step 5
	median kurtosis: 12.018216133117676
	20th percentile R2(t,t-1): 0.9613077640533447
	taking step 6
	median kurtosis: 9.285207748413086
	20th percentile R2(t,t-1): 0.9742846488952637
stopping after 6 steps
covariate-adjusted NAM not found; computing and saving


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing SVD
performing association test
computing neighborhood-level FDRs
18
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.684931506849315 , 18 PCs used
total r^2 between top 18 NAM PCs and outcome is 0.12


In [23]:
sc_uni['Sex'] = res.p

# RESPONSE

## Single Cell

In [36]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/age_sex_response_race/sc_meta_response.csv')
print(meta.shape)

(90839, 37)


In [37]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/age_sex_response_race/sc_harmony_response.csv')
print(harmony.shape)

(90839, 20)


In [38]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/age_sex_response_race/sc_umap_response.csv')
print(umap.shape)

(90839, 2)


In [39]:
# Start by making anndata object
d = mad.MultiAnnData(X=harmony, obs=meta, sampleid="sample")
#d.obs_to_sample(['Type', 'Site',
#                 'Responder_Status', 'Sex',
#                 'Age', 'Race',
#                'Ethnicity', 'ISN',
#               'Activity', "Chronicity",
#                 'processing_batch'])
d.obs_to_sample(['Sex', 'Age', 'Final_Chronicity', 'Final_Activity', 'First_biop', 'Pred_use', 'Responder_Status', 'Race_[A]',
       'Race_[A][B]', 'Race_[B]', 'Race_[B][AI]', 'Race_[U]',
       'Race_[W]', 'Final_ISN_[III]', 'Final_ISN_[III][V]', 'Final_ISN_[IV]',
       'Final_ISN_[IV][V]', 'Final_ISN_[V]',
       'Final_Site_Einstein', 'Final_Site_JHU', 'Final_Site_Michigan',
       'Final_Site_MUSC', 'Final_Site_Northwell', 'Final_Site_NYU',
       'Final_Site_Rochester', 'Final_Site_Texas Tech', 
       'Final_Site_UCSD', 'Final_Site_UCSF', 'injured_pt_prop'])
d.samplem.head()

/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


['cell' 'sample' 'cluster_names' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies


,Sex,Age,Final_Chronicity,Final_Activity,First_biop,Pred_use,Responder_Status,Race_[A],Race_[A][B],Race_[B],...,Final_Site_JHU,Final_Site_Michigan,Final_Site_MUSC,Final_Site_Northwell,Final_Site_NYU,Final_Site_Rochester,Final_Site_Texas Tech,Final_Site_UCSD,Final_Site_UCSF,injured_pt_prop
sample,,,,,,,,,,,,,,,,,,,,,
AMPSLEkid_cells_0373,1.0,-0.257482,5.0,3.0,0.0,1.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,9.442231
AMPSLEkid_cells_0964,1.0,-1.220110,3.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,8.129730
AMPSLEkid_cells_1135,1.0,-1.307621,3.0,7.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,8.852868
AMPSLEkid_cells_1154,1.0,0.792657,3.0,7.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4.682512
AMPSLEkid_cells_1226,1.0,0.705146,3.0,4.0,0.0,1.0,1.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.319940


In [40]:
umap.index = d.obs.index
d.obsm['X_umap'] = umap

In [41]:
np.random.seed(0) 
cna.pp.knn(d)

computing default knn graph


In [42]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Responder_Status, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 42.608909606933594
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 28.298498153686523
	20th percentile R2(t,t-1): 0.7690775394439697
	taking step 3
	median kurtosis: 19.13600730895996
	20th percentile R2(t,t-1): 0.91077721118927
	taking step 4
	median kurtosis: 13.476375579833984
	20th percentile R2(t,t-1): 0.9449318647384644
	taking step 5
	median kurtosis: 10.11384105682373
	20th percentile R2(t,t-1): 0.9633954167366028
	taking step 6
	median kurtosis: 8.06271743774414
	20th percentile R2(t,t-1): 0.9760456085205078
stopping after 6 steps
covariate-adjusted NAM not found; computing and saving


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing SVD
performing association test
computing neighborhood-level FDRs
1
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.028897110288971104 , 1 PCs used
total r^2 between top 1 NAM PCs and outcome is 0.07


In [43]:
response_uni = res.p

In [44]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Responder_Status, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      covs = d.samplem[['Final_Chronicity']],
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

covariate-adjusted NAM not found; computing and saving


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing SVD
performing association test
computing neighborhood-level FDRs
2
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.6942305769423057 , 2 PCs used
total r^2 between top 2 NAM PCs and outcome is 0.03


In [45]:
response_cond = res.p

In [34]:
sc_uni['Responder_Status'] = res.p

In [35]:
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/age_sex_response_race/sc_response_ncorr.csv", 
               res.ncorrs, delimiter=",")
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/age_sex_response_race/sc_response_fdrs.csv", 
               res.fdrs, delimiter=",")

In [23]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Responder_Status, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      covs = d.samplem[['Final_Chronicity', 'First_biop', 'Pred_use']],
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 42.608909606933594
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 28.298498153686523
	20th percentile R2(t,t-1): 0.7690775394439697
	taking step 3
	median kurtosis: 19.13600730895996
	20th percentile R2(t,t-1): 0.91077721118927
	taking step 4
	median kurtosis: 13.476375579833984
	20th percentile R2(t,t-1): 0.9449318647384644
	taking step 5
	median kurtosis: 10.11384105682373
	20th percentile R2(t,t-1): 0.9633954167366028
	taking step 6
	median kurtosis: 8.06271743774414
	20th percentile R2(t,t-1): 0.9760456085205078
stopping after 6 steps
covariate-adjusted NAM not found; computing and saving


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing SVD
performing association test
computing neighborhood-level FDRs
1
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.2628737126287371 , 1 PCs used
total r^2 between top 1 NAM PCs and outcome is 0.04


In [27]:
sc_cond['Responder_Status'] = res.p

In [9]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Responder_Status, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      covs = d.samplem[['injured_pt_prop']],
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

covariate-adjusted NAM not found; computing and saving


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing SVD
performing association test
computing neighborhood-level FDRs
1
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.43765623437656237 , 1 PCs used
total r^2 between top 1 NAM PCs and outcome is 0.02


In [15]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.injured_pt_prop, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                 #     covs = d.samplem[['Responder_Status']],
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
12
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 9.999000099990002e-05 , 12 PCs used
total r^2 between top 12 NAM PCs and outcome is 0.59


In [17]:
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/age_sex_response_race/sc_injured_pt_ncorr.csv", 
               res.ncorrs, delimiter=",")
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/age_sex_response_race/sc_injured_pt_fdrs.csv", 
               res.fdrs, delimiter=",")

In [46]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Chronicity, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

covariate-adjusted NAM not found; computing and saving


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing SVD
performing association test


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_association.py:74: UserWarning: global association p-value attained minimal possible value. Consider increasing Nnull
  warnings.warn('global association p-value attained minimal possible value. '+\


computing neighborhood-level FDRs
8
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 9.999000099990002e-05 , 8 PCs used
total r^2 between top 8 NAM PCs and outcome is 0.59


In [47]:
chron_uni = res.p

In [48]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Chronicity, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      covs = d.samplem[['Responder_Status']],
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

covariate-adjusted NAM not found; computing and saving


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing SVD
performing association test


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_association.py:74: UserWarning: global association p-value attained minimal possible value. Consider increasing Nnull
  warnings.warn('global association p-value attained minimal possible value. '+\


computing neighborhood-level FDRs
8
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 9.999000099990002e-05 , 8 PCs used
total r^2 between top 8 NAM PCs and outcome is 0.57


In [49]:
chron_cond = res.p

In [50]:
pd.DataFrame(dict(pval = [response_uni, response_cond, chron_uni, chron_cond],
             variable = ['Responder Status', 'Responder Status', 'Chronicity', 'Chronicity'],
             test = ['Univariate', 'Conditional', 'Univariate', 'Conditional'],
             cell_type = ['LOH', 'LOH', 'LOH', 'LOH'])).to_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/age_sex_response_race/response_chronicity_cond.csv')

## Single Nuclei

In [39]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/age_sex_response_race/sn_meta_response.csv')
print(meta.shape)

(34949, 10)


In [40]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/age_sex_response_race/sn_harmony_response.csv')
print(harmony.shape)

(34949, 20)


In [41]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/age_sex_response_race/sn_umap_response.csv')
print(umap.shape)

(34949, 2)


In [42]:
# Start by making anndata object
d = mad.MultiAnnData(X=harmony, obs=meta, sampleid="sample")
#d.obs_to_sample(['Type', 'Site',
#                 'Responder_Status', 'Sex',
#                 'Age', 'Race',
#                'Ethnicity', 'ISN',
#               'Activity', "Chronicity",
#                 'processing_batch'])
d.obs_to_sample(['Responder_Status'])
d.samplem.head()

['cell' 'Sex' 'sample' 'cluster_names' 'Responder.Status' 'Race' 'ISN'
 'Type']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


,Responder_Status
sample,
AMPSLEkid_cells_0147,1.0
AMPSLEkid_cells_0366,0.0
AMPSLEkid_cells_0849,0.0
AMPSLEkid_cells_0880,0.0
AMPSLEkid_cells_0883,0.0


In [43]:
umap.index = d.obs.index
d.obsm['X_umap'] = umap

In [44]:
np.random.seed(0) 
cna.pp.knn(d)

computing default knn graph


In [45]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Responder_Status, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 11.70981216430664
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 8.217973709106445
	20th percentile R2(t,t-1): 0.7747018337249756
	taking step 3
	median kurtosis: 5.639926910400391
	20th percentile R2(t,t-1): 0.9109618544578553
stopping after 3 steps
covariate-adjusted NAM not found; computing and saving
computing SVD
performing association test


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing neighborhood-level FDRs
19
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.0973902609739026 , 19 PCs used
total r^2 between top 19 NAM PCs and outcome is 0.65


In [46]:
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/age_sex_response_race/sn_response_ncorr.csv", 
               res.ncorrs, delimiter=",")
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/age_sex_response_race/sn_response_fdrs.csv", 
               res.fdrs, delimiter=",")

# RACE

## Single Cell

In [43]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/age_sex_response_race/sc_meta_race.csv')
print(meta.shape)

(116401, 39)


In [44]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/age_sex_response_race/sc_harmony_race.csv')
print(harmony.shape)

(116401, 20)


In [45]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/age_sex_response_race/sc_umap_race.csv')
print(umap.shape)

(116401, 2)


In [46]:
# Start by making anndata object
d = mad.MultiAnnData(X=harmony, obs=meta, sampleid="sample")
#d.obs_to_sample(['Type', 'Site',
#                 'Responder_Status', 'Sex',
#                 'Age', 'Race',
#                'Ethnicity', 'ISN',
#               'Activity', "Chronicity",
#                 'processing_batch'])
d.obs_to_sample(['Sex', 'Age', 'Final_Chronicity', 'Final_Activity', 'First_biop', 'Pred_use', 'Responder_Status', 'Race_[A]',
       'Race_[A][B]', 'Race_[B]', 'Race_[B][AI]', 'Race_[B][W]', 'Race_[U]',
       'Race_[W]', 'Final_ISN_[III]', 'Final_ISN_[III][V]', 'Final_ISN_[IV]',
       'Final_ISN_[IV][V]', 'Final_ISN_[V]', 'Final_Site_Cincinnati',
       'Final_Site_Einstein', 'Final_Site_JHU', 'Final_Site_Michigan',
       'Final_Site_MUSC', 'Final_Site_Northwell', 'Final_Site_NYU',
       'Final_Site_Rochester', 'Final_Site_Texas Tech', 'Final_Site_UCLA',
       'Final_Site_UCSD', 'Final_Site_UCSF'])
d.samplem.head()

/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


['cell' 'sample' 'cluster_names' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies


,Sex,Age,Final_Chronicity,Final_Activity,First_biop,Pred_use,Responder_Status,Race_[A],Race_[A][B],Race_[B],...,Final_Site_JHU,Final_Site_Michigan,Final_Site_MUSC,Final_Site_Northwell,Final_Site_NYU,Final_Site_Rochester,Final_Site_Texas Tech,Final_Site_UCLA,Final_Site_UCSD,Final_Site_UCSF
sample,,,,,,,,,,,,,,,,,,,,,
AMPSLEkid_cells_0373,1.0,-0.316264,5.0,3.0,0.0,1.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
AMPSLEkid_cells_0382,1.0,1.904141,3.0,8.0,1.0,1.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
AMPSLEkid_cells_0964,1.0,-1.255666,3.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
AMPSLEkid_cells_1135,1.0,-1.341067,3.0,7.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_1154,1.0,0.708538,3.0,7.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [47]:
umap.index = d.obs.index
d.obsm['X_umap'] = umap

In [48]:
np.random.seed(0) 
cna.pp.knn(d)

computing default knn graph


In [55]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem['Race_[A]'], #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 53.41489028930664
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 35.2132568359375
	20th percentile R2(t,t-1): 0.7658944725990295
	taking step 3
	median kurtosis: 23.83907127380371
	20th percentile R2(t,t-1): 0.9089255332946777
	taking step 4
	median kurtosis: 16.483577728271484
	20th percentile R2(t,t-1): 0.9428997039794922
	taking step 5
	median kurtosis: 12.018216133117676
	20th percentile R2(t,t-1): 0.9613077640533447
	taking step 6
	median kurtosis: 9.285207748413086
	20th percentile R2(t,t-1): 0.9742846488952637
stopping after 6 steps
covariate-adjusted NAM not found; computing and saving


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing SVD
performing association test
computing neighborhood-level FDRs
1
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.5488451154884512 , 1 PCs used
total r^2 between top 1 NAM PCs and outcome is 0.01


In [56]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem['Race_[A][B]'], #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
3
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.14508549145085492 , 3 PCs used
total r^2 between top 3 NAM PCs and outcome is 0.08


In [57]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem['Race_[B]'], #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
2
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.1825817418258174 , 2 PCs used
total r^2 between top 2 NAM PCs and outcome is 0.04


In [58]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem['Race_[B][AI]'], #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
2
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.8669133086691331 , 2 PCs used
total r^2 between top 2 NAM PCs and outcome is 0.01


In [59]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem['Race_[B][W]'], #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
6
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.2636736326367363 , 6 PCs used
total r^2 between top 6 NAM PCs and outcome is 0.07


In [60]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem['Race_[U]'], #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
11
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.20037996200379962 , 11 PCs used
total r^2 between top 11 NAM PCs and outcome is 0.13


In [49]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem['Race_[W]'], #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 53.41489028930664
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 35.2132568359375
	20th percentile R2(t,t-1): 0.7658944725990295
	taking step 3
	median kurtosis: 23.83907127380371
	20th percentile R2(t,t-1): 0.9089255332946777
	taking step 4
	median kurtosis: 16.483577728271484
	20th percentile R2(t,t-1): 0.9428997039794922
	taking step 5
	median kurtosis: 12.018216133117676
	20th percentile R2(t,t-1): 0.9613077640533447
	taking step 6
	median kurtosis: 9.285207748413086
	20th percentile R2(t,t-1): 0.9742846488952637
stopping after 6 steps
covariate-adjusted NAM not found; computing and saving


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing SVD
performing association test
computing neighborhood-level FDRs
10
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.0922907709229077 , 10 PCs used
total r^2 between top 10 NAM PCs and outcome is 0.13


In [50]:
sc_uni['Race'] = res.p

## Single Nuclear

In [39]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/age_sex_response_race/sn_meta_race.csv')
print(meta.shape)

(36541, 33)


In [40]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/age_sex_response_race/sn_harmony_race.csv')
print(harmony.shape)

(36541, 20)


In [41]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/age_sex_response_race/sn_umap_race.csv')
print(umap.shape)

(36541, 2)


In [48]:
# Start by making anndata object
d = mad.MultiAnnData(X=harmony, obs=meta, sampleid="sample")
#d.obs_to_sample(['Type', 'Site',
#                 'Responder_Status', 'Sex',
#                 'Age', 'Race',
#                'Ethnicity', 'ISN',
#               'Activity', "Chronicity",
#                 'processing_batch'])
d.obs_to_sample(['Sex', 'Age', 'Final_Chronicity', 'Final_Activity', 'First_biop', 'Pred_use', 'Responder_Status', 'Race_[A]',
       'Race_[B]', 'Race_[U]',
       'Race_[W]', 'Final_ISN_[III]', 'Final_ISN_[III][V]', 'Final_ISN_[IV]',
       'Final_ISN_[IV][V]', 'Final_ISN_[V]', 'Final_Site_Cincinnati',
        'Final_Site_JHU', 'Final_Site_Michigan',
       'Final_Site_MUSC', 'Final_Site_Northwell', 'Final_Site_NYU',
       'Final_Site_Rochester', 'Final_Site_Texas Tech', 
        'Final_Site_UCSF'])
d.samplem.head()

['cell' 'sample' 'cluster_names' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


,Sex,Age,Final_Chronicity,Final_Activity,First_biop,Pred_use,Responder_Status,Race_[A],Race_[B],Race_[U],...,Final_ISN_[V],Final_Site_Cincinnati,Final_Site_JHU,Final_Site_Michigan,Final_Site_MUSC,Final_Site_Northwell,Final_Site_NYU,Final_Site_Rochester,Final_Site_Texas Tech,Final_Site_UCSF
sample,,,,,,,,,,,,,,,,,,,,,
AMPSLEkid_cells_0147,1.0,0.788549,9.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
AMPSLEkid_cells_0366,2.0,-1.232745,7.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
AMPSLEkid_cells_0849,1.0,0.237287,9.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0880,1.0,0.880426,NaN,NaN,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0883,2.0,-1.416499,NaN,NaN,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [49]:
umap.index = d.obs.index
d.obsm['X_umap'] = umap

In [50]:
np.random.seed(0) 
cna.pp.knn(d)

computing default knn graph


In [55]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem['Race_[A]'], #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
12
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.792020797920208 , 12 PCs used
total r^2 between top 12 NAM PCs and outcome is 0.33


In [52]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem['Race_[B]'], #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
12
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.029697030296970302 , 12 PCs used
total r^2 between top 12 NAM PCs and outcome is 0.58


In [53]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem['Race_[U]'], #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
7
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.8913108689131087 , 7 PCs used
total r^2 between top 7 NAM PCs and outcome is 0.17


In [54]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem['Race_[W]'], #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
5
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.005699430056994301 , 5 PCs used
total r^2 between top 5 NAM PCs and outcome is 0.49


# SITE

## Single Cell

In [10]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/first_biop_pred_use_site/sc_meta_site.csv')
print(meta.shape)

(116401, 39)


In [11]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/first_biop_pred_use_site/sc_harmony_site.csv')
print(harmony.shape)

(116401, 20)


In [12]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/first_biop_pred_use_site/sc_umap_site.csv')
print(umap.shape)

(116401, 2)


In [32]:
# Start by making anndata object
d = mad.MultiAnnData(X=harmony, obs=meta, sampleid="sample")
#d.obs_to_sample(['Type', 'Site',
#                 'Responder_Status', 'Sex',
#                 'Age', 'Race',
#                'Ethnicity', 'ISN',
#               'Activity', "Chronicity",
#                 'processing_batch'])
d.obs_to_sample(['Sex', 'Age', 'Final_Chronicity', 'Final_Activity', 'First_biop', 'Pred_use', 'Responder_Status', 'Race_[A]',
       'Race_[A][B]', 'Race_[B]', 'Race_[B][AI]', 'Race_[B][W]', 'Race_[U]',
       'Race_[W]', 'Final_ISN_[III]', 'Final_ISN_[III][V]', 'Final_ISN_[IV]',
       'Final_ISN_[IV][V]', 'Final_ISN_[V]', 'Final_Site_Cincinnati',
       'Final_Site_Einstein', 'Final_Site_JHU', 'Final_Site_Michigan',
       'Final_Site_MUSC', 'Final_Site_Northwell', 'Final_Site_NYU',
       'Final_Site_Rochester', 'Final_Site_Texas Tech', 'Final_Site_UCLA',
       'Final_Site_UCSD', 'Final_Site_UCSF'])
d.samplem.head()

/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


['cell' 'sample' 'cluster_names' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies


,Sex,Age,Final_Chronicity,Final_Activity,First_biop,Pred_use,Responder_Status,Race_[A],Race_[A][B],Race_[B],...,Final_Site_JHU,Final_Site_Michigan,Final_Site_MUSC,Final_Site_Northwell,Final_Site_NYU,Final_Site_Rochester,Final_Site_Texas Tech,Final_Site_UCLA,Final_Site_UCSD,Final_Site_UCSF
sample,,,,,,,,,,,,,,,,,,,,,
AMPSLEkid_cells_0373,1.0,-0.316264,5.0,3.0,0.0,1.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
AMPSLEkid_cells_0382,1.0,1.904141,3.0,8.0,1.0,1.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
AMPSLEkid_cells_0964,1.0,-1.255666,3.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
AMPSLEkid_cells_1135,1.0,-1.341067,3.0,7.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_1154,1.0,0.708538,3.0,7.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [33]:
umap.index = d.obs.index
d.obsm['X_umap'] = umap

In [34]:
np.random.seed(0) 
cna.pp.knn(d)

computing default knn graph


In [57]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Site_Cincinnati, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 53.41489028930664
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 35.2132568359375
	20th percentile R2(t,t-1): 0.7658944725990295
	taking step 3
	median kurtosis: 23.83907127380371
	20th percentile R2(t,t-1): 0.9089255332946777
	taking step 4
	median kurtosis: 16.483577728271484
	20th percentile R2(t,t-1): 0.9428997039794922
	taking step 5
	median kurtosis: 12.018216133117676
	20th percentile R2(t,t-1): 0.9613077640533447
	taking step 6
	median kurtosis: 9.285207748413086
	20th percentile R2(t,t-1): 0.9742846488952637
stopping after 6 steps
covariate-adjusted NAM not found; computing and saving


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing SVD
performing association test
computing neighborhood-level FDRs
1
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.39856014398560147 , 1 PCs used
total r^2 between top 1 NAM PCs and outcome is 0.01


In [70]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Site_Einstein, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
10
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.0458954104589541 , 10 PCs used
total r^2 between top 10 NAM PCs and outcome is 0.18


In [71]:
sc_uni['Site'] = res.p

In [35]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Site_Einstein, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      covs = d.samplem[['Final_Chronicity', 'Final_Activity', 
                                                        'First_biop', 'Pred_use', 'Responder_Status',
                                                        'Final_ISN_[III][V]', 'Final_ISN_[IV]',
                                                        'Final_ISN_[IV][V]', 'Final_ISN_[V]']], 
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 53.41489028930664
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 35.2132568359375
	20th percentile R2(t,t-1): 0.7658944725990295
	taking step 3
	median kurtosis: 23.83907127380371
	20th percentile R2(t,t-1): 0.9089255332946777
	taking step 4
	median kurtosis: 16.483577728271484
	20th percentile R2(t,t-1): 0.9428997039794922
	taking step 5
	median kurtosis: 12.018216133117676
	20th percentile R2(t,t-1): 0.9613077640533447
	taking step 6
	median kurtosis: 9.285207748413086
	20th percentile R2(t,t-1): 0.9742846488952637
stopping after 6 steps
covariate-adjusted NAM not found; computing and saving


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing SVD
performing association test


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_association.py:57: UserWarning: data supported use of 20 NAM PCs, which is the maximum considered. Consider allowing more PCs by using the "ks" argument.
  warnings.warn(('data supported use of {} NAM PCs, which is the maximum considered. '+\


computing neighborhood-level FDRs
20
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.013898610138986101 , 20 PCs used
total r^2 between top 20 NAM PCs and outcome is 0.42


In [36]:
sc_cond['Site'] = res.p

In [60]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Site_JHU, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
20
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.20427957204279573 , 20 PCs used
total r^2 between top 20 NAM PCs and outcome is 0.18


In [61]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Site_Michigan, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
4
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.7862213778622138 , 4 PCs used
total r^2 between top 4 NAM PCs and outcome is 0.02


In [62]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Site_MUSC, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
1
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.2084791520847915 , 1 PCs used
total r^2 between top 1 NAM PCs and outcome is 0.04


In [63]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Site_Northwell, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
2
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.9819018098190181 , 2 PCs used
total r^2 between top 2 NAM PCs and outcome is 0.00


In [64]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Site_NYU, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
9
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.3023697630236976 , 9 PCs used
total r^2 between top 9 NAM PCs and outcome is 0.10


In [65]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Site_Rochester, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
5
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.35966403359664034 , 5 PCs used
total r^2 between top 5 NAM PCs and outcome is 0.06


In [66]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem['Final_Site_Texas Tech'], #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
20
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.07579242075792421 , 20 PCs used
total r^2 between top 20 NAM PCs and outcome is 0.33


In [67]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Site_UCLA, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
1
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.24177582241775822 , 1 PCs used
total r^2 between top 1 NAM PCs and outcome is 0.02


In [68]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Site_UCSD, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
1
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.946005399460054 , 1 PCs used
total r^2 between top 1 NAM PCs and outcome is 0.00


In [69]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Site_UCSF, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
1
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.8515148485151485 , 1 PCs used
total r^2 between top 1 NAM PCs and outcome is 0.00


## Single Nuclear

In [13]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/first_biop_pred_use_site/sn_meta_site.csv')
print(meta.shape)

(36541, 33)


In [14]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/first_biop_pred_use_site/sn_harmony_site.csv')
print(harmony.shape)

(36541, 20)


In [17]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/first_biop_pred_use_site/sn_umap_site.csv')
print(umap.shape)

(36541, 2)


In [18]:
# Start by making anndata object
d = mad.MultiAnnData(X=harmony, obs=meta, sampleid="sample")
#d.obs_to_sample(['Type', 'Site',
#                 'Responder_Status', 'Sex',
#                 'Age', 'Race',
#                'Ethnicity', 'ISN',
#               'Activity', "Chronicity",
#                 'processing_batch'])
d.obs_to_sample(['Sex', 'Age', 'Final_Chronicity', 'Final_Activity', 'First_biop', 'Pred_use', 'Responder_Status', 'Race_[A]',
       'Race_[B]', 'Race_[U]',
       'Race_[W]', 'Final_ISN_[III]', 'Final_ISN_[III][V]', 'Final_ISN_[IV]',
       'Final_ISN_[IV][V]', 'Final_ISN_[V]', 'Final_Site_Cincinnati',
       'Final_Site_JHU', 'Final_Site_Michigan',
       'Final_Site_MUSC', 'Final_Site_Northwell', 'Final_Site_NYU',
       'Final_Site_Rochester', 'Final_Site_Texas Tech', 
       'Final_Site_UCSF'])
d.samplem.head()

['cell' 'sample' 'cluster_names' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


,Sex,Age,Final_Chronicity,Final_Activity,First_biop,Pred_use,Responder_Status,Race_[A],Race_[B],Race_[U],...,Final_ISN_[V],Final_Site_Cincinnati,Final_Site_JHU,Final_Site_Michigan,Final_Site_MUSC,Final_Site_Northwell,Final_Site_NYU,Final_Site_Rochester,Final_Site_Texas Tech,Final_Site_UCSF
sample,,,,,,,,,,,,,,,,,,,,,
AMPSLEkid_cells_0147,1.0,0.788549,9.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
AMPSLEkid_cells_0366,2.0,-1.232745,7.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
AMPSLEkid_cells_0849,1.0,0.237287,9.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0880,1.0,0.880426,NaN,NaN,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0883,2.0,-1.416499,NaN,NaN,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [19]:
umap.index = d.obs.index
d.obsm['X_umap'] = umap

In [20]:
np.random.seed(0) 
cna.pp.knn(d)

computing default knn graph


In [21]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Site_Cincinnati, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 12.009225845336914
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 8.349165916442871
	20th percentile R2(t,t-1): 0.7731621861457825
	taking step 3
	median kurtosis: 5.703102111816406
	20th percentile R2(t,t-1): 0.9103659629821778
stopping after 3 steps
covariate-adjusted NAM not found; computing and saving
computing SVD
performing association test


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing neighborhood-level FDRs
1
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.5394460553944606 , 1 PCs used
total r^2 between top 1 NAM PCs and outcome is 0.05


In [22]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Site_JHU, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
18
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.6441355864413558 , 18 PCs used
total r^2 between top 18 NAM PCs and outcome is 0.47


In [23]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Site_Michigan, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
2
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.8761123887611239 , 2 PCs used
total r^2 between top 2 NAM PCs and outcome is 0.03


In [25]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Site_MUSC, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_association.py:57: UserWarning: data supported use of 20 NAM PCs, which is the maximum considered. Consider allowing more PCs by using the "ks" argument.
  warnings.warn(('data supported use of {} NAM PCs, which is the maximum considered. '+\


computing neighborhood-level FDRs
20
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.37236276372362764 , 20 PCs used
total r^2 between top 20 NAM PCs and outcome is 0.67


In [26]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Site_Northwell, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
4
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.6997300269973002 , 4 PCs used
total r^2 between top 4 NAM PCs and outcome is 0.18


In [28]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Site_NYU, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
15
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.5263473652634737 , 15 PCs used
total r^2 between top 15 NAM PCs and outcome is 0.45


In [29]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Site_Rochester, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
20
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.2999700029997 , 20 PCs used
total r^2 between top 20 NAM PCs and outcome is 0.62


In [30]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem['Final_Site_Texas Tech'], #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
19
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.22297770222977703 , 19 PCs used
total r^2 between top 19 NAM PCs and outcome is 0.86


In [31]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Site_UCSF, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
19
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.22297770222977703 , 19 PCs used
total r^2 between top 19 NAM PCs and outcome is 0.71


# FIRST BIOPSY

## Single Cell

In [62]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/first_biop_pred_use_site/sc_meta_firstbiop.csv')
print(meta.shape)

(116401, 39)


In [63]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/first_biop_pred_use_site/sc_harmony_firstbiop.csv')
print(harmony.shape)

(116401, 20)


In [64]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/first_biop_pred_use_site/sc_umap_firstbiop.csv')
print(umap.shape)

(116401, 2)


In [59]:
# Start by making anndata object
d = mad.MultiAnnData(X=harmony, obs=meta, sampleid="sample")
#d.obs_to_sample(['Type', 'Site',
#                 'Responder_Status', 'Sex',
#                 'Age', 'Race',
#                'Ethnicity', 'ISN',
#               'Activity', "Chronicity",
#                 'processing_batch'])
d.obs_to_sample(['Sex', 'Age', 'Final_Chronicity', 'Final_Activity', 'First_biop', 'Pred_use', 'Responder_Status', 'Race_[A]',
       'Race_[A][B]', 'Race_[B]', 'Race_[B][AI]', 'Race_[B][W]', 'Race_[U]',
       'Race_[W]', 'Final_ISN_[III]', 'Final_ISN_[III][V]', 'Final_ISN_[IV]',
       'Final_ISN_[IV][V]', 'Final_ISN_[V]', 'Final_Site_Cincinnati',
       'Final_Site_Einstein', 'Final_Site_JHU', 'Final_Site_Michigan',
       'Final_Site_MUSC', 'Final_Site_Northwell', 'Final_Site_NYU',
       'Final_Site_Rochester', 'Final_Site_Texas Tech', 'Final_Site_UCLA',
       'Final_Site_UCSD', 'Final_Site_UCSF'])
d.samplem.head()

['cell' 'sample' 'cluster_names' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


,Sex,Age,Final_Chronicity,Final_Activity,First_biop,Pred_use,Responder_Status,Race_[A],Race_[A][B],Race_[B],...,Final_Site_JHU,Final_Site_Michigan,Final_Site_MUSC,Final_Site_Northwell,Final_Site_NYU,Final_Site_Rochester,Final_Site_Texas Tech,Final_Site_UCLA,Final_Site_UCSD,Final_Site_UCSF
sample,,,,,,,,,,,,,,,,,,,,,
AMPSLEkid_cells_0373,1.0,-0.316264,5.0,3.0,0.0,1.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
AMPSLEkid_cells_0382,1.0,1.904141,3.0,8.0,1.0,1.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
AMPSLEkid_cells_0964,1.0,-1.255666,3.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
AMPSLEkid_cells_1135,1.0,-1.341067,3.0,7.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_1154,1.0,0.708538,3.0,7.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [60]:
umap.index = d.obs.index
d.obsm['X_umap'] = umap

In [61]:
np.random.seed(0) 
cna.pp.knn(d)

computing default knn graph


In [78]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.First_biop, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 53.41489028930664
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 35.2132568359375
	20th percentile R2(t,t-1): 0.7658944725990295
	taking step 3
	median kurtosis: 23.83907127380371
	20th percentile R2(t,t-1): 0.9089255332946777
	taking step 4
	median kurtosis: 16.483577728271484
	20th percentile R2(t,t-1): 0.9428997039794922
	taking step 5
	median kurtosis: 12.018216133117676
	20th percentile R2(t,t-1): 0.9613077640533447
	taking step 6
	median kurtosis: 9.285207748413086
	20th percentile R2(t,t-1): 0.9742846488952637
stopping after 6 steps
covariate-adjusted NAM not found; computing and saving


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing SVD
performing association test
computing neighborhood-level FDRs
15
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.0005999400059994001 , 15 PCs used
total r^2 between top 15 NAM PCs and outcome is 0.25


In [80]:
sc_uni['First_biopsy'] = res.p

In [66]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.First_biop, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      covs = d.samplem[['Final_Chronicity', 'Responder_Status',
                                                        'Final_ISN_[III][V]', 'Final_ISN_[IV]',
                                                        'Final_ISN_[IV][V]', 'Final_ISN_[V]',
                                                        'Pred_use',
                                                        'Final_Site_Einstein', 'Final_Site_JHU', 
                                                        'Final_Site_MUSC', 'Final_Site_NYU',
                                                        'Final_Site_Rochester', 'Final_Site_Texas Tech', 
                                                        'Final_Site_UCSD', 'Final_Site_UCSF']],
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

covariate-adjusted NAM not found; computing and saving


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing SVD
performing association test
computing neighborhood-level FDRs
19
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.0034996500349965005 , 19 PCs used
total r^2 between top 19 NAM PCs and outcome is 0.38


In [67]:
sc_cond['First_biopsy'] = res.p

## Single Nuclear

In [71]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/first_biop_pred_use_site/sn_meta_firstbiop.csv')
print(meta.shape)

(36541, 33)


In [72]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/first_biop_pred_use_site/sn_harmony_firstbiop.csv')
print(harmony.shape)

(36541, 20)


In [73]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/first_biop_pred_use_site/sn_umap_firstbiop.csv')
print(umap.shape)

(36541, 2)


In [74]:
# Start by making anndata object
d = mad.MultiAnnData(X=harmony, obs=meta, sampleid="sample")
#d.obs_to_sample(['Type', 'Site',
#                 'Responder_Status', 'Sex',
#                 'Age', 'Race',
#                'Ethnicity', 'ISN',
#               'Activity', "Chronicity",
#                 'processing_batch'])
d.obs_to_sample(['Sex', 'Age', 'Final_Chronicity', 'Final_Activity', 'First_biop', 'Pred_use', 'Responder_Status', 'Race_[A]',
       'Race_[B]', 'Race_[U]',
       'Race_[W]', 'Final_ISN_[III]', 'Final_ISN_[III][V]', 'Final_ISN_[IV]',
       'Final_ISN_[IV][V]', 'Final_ISN_[V]', 'Final_Site_Cincinnati',
       'Final_Site_JHU', 'Final_Site_Michigan',
       'Final_Site_MUSC', 'Final_Site_Northwell', 'Final_Site_NYU',
       'Final_Site_Rochester', 'Final_Site_Texas Tech', 
       'Final_Site_UCSF'])
d.samplem.head()

['cell' 'sample' 'cluster_names' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


,Sex,Age,Final_Chronicity,Final_Activity,First_biop,Pred_use,Responder_Status,Race_[A],Race_[B],Race_[U],...,Final_ISN_[V],Final_Site_Cincinnati,Final_Site_JHU,Final_Site_Michigan,Final_Site_MUSC,Final_Site_Northwell,Final_Site_NYU,Final_Site_Rochester,Final_Site_Texas Tech,Final_Site_UCSF
sample,,,,,,,,,,,,,,,,,,,,,
AMPSLEkid_cells_0147,1.0,0.788549,9.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
AMPSLEkid_cells_0366,2.0,-1.232745,7.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
AMPSLEkid_cells_0849,1.0,0.237287,9.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0880,1.0,0.880426,NaN,NaN,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0883,2.0,-1.416499,NaN,NaN,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [75]:
umap.index = d.obs.index
d.obsm['X_umap'] = umap

In [76]:
np.random.seed(0) 
cna.pp.knn(d)

computing default knn graph


In [77]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.First_biop, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 12.009225845336914
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 8.349165916442871
	20th percentile R2(t,t-1): 0.7731621861457825
	taking step 3
	median kurtosis: 5.703102111816406
	20th percentile R2(t,t-1): 0.9103659629821778
stopping after 3 steps
covariate-adjusted NAM not found; computing and saving
computing SVD
performing association test


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing neighborhood-level FDRs
17
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.004899510048995101 , 17 PCs used
total r^2 between top 17 NAM PCs and outcome is 0.74


# PREDNISONE USE

## Single Cell

In [4]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/first_biop_pred_use_site/sc_meta_preduse.csv')
print(meta.shape)

(116401, 39)


In [5]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/first_biop_pred_use_site/sc_harmony_preduse.csv')
print(harmony.shape)

(116401, 20)


In [6]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/first_biop_pred_use_site/sc_umap_preduse.csv')
print(umap.shape)

(116401, 2)


In [7]:
# Start by making anndata object
d = mad.MultiAnnData(X=harmony, obs=meta, sampleid="sample")
#d.obs_to_sample(['Type', 'Site',
#                 'Responder_Status', 'Sex',
#                 'Age', 'Race',
#                'Ethnicity', 'ISN',
#               'Activity', "Chronicity",
#                 'processing_batch'])
d.obs_to_sample(['Sex', 'Age', 'Final_Chronicity', 'Final_Activity', 'First_biop', 'Pred_use', 'Responder_Status', 'Race_[A]',
       'Race_[A][B]', 'Race_[B]', 'Race_[B][AI]', 'Race_[B][W]', 'Race_[U]',
       'Race_[W]', 'Final_ISN_[III]', 'Final_ISN_[III][V]', 'Final_ISN_[IV]',
       'Final_ISN_[IV][V]', 'Final_ISN_[V]', 'Final_Site_Cincinnati',
       'Final_Site_Einstein', 'Final_Site_JHU', 'Final_Site_Michigan',
       'Final_Site_MUSC', 'Final_Site_Northwell', 'Final_Site_NYU',
       'Final_Site_Rochester', 'Final_Site_Texas Tech', 'Final_Site_UCLA',
       'Final_Site_UCSD', 'Final_Site_UCSF'])
d.samplem.head()

/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


['cell' 'sample' 'cluster_names' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies


,Sex,Age,Final_Chronicity,Final_Activity,First_biop,Pred_use,Responder_Status,Race_[A],Race_[A][B],Race_[B],...,Final_Site_JHU,Final_Site_Michigan,Final_Site_MUSC,Final_Site_Northwell,Final_Site_NYU,Final_Site_Rochester,Final_Site_Texas Tech,Final_Site_UCLA,Final_Site_UCSD,Final_Site_UCSF
sample,,,,,,,,,,,,,,,,,,,,,
AMPSLEkid_cells_0373,1.0,-0.316264,5.0,3.0,0.0,1.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
AMPSLEkid_cells_0382,1.0,1.904141,3.0,8.0,1.0,1.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
AMPSLEkid_cells_0964,1.0,-1.255666,3.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
AMPSLEkid_cells_1135,1.0,-1.341067,3.0,7.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_1154,1.0,0.708538,3.0,7.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [8]:
umap.index = d.obs.index
d.obsm['X_umap'] = umap

In [9]:
np.random.seed(0) 
cna.pp.knn(d)

computing default knn graph


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [87]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Pred_use, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 53.41489028930664
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 35.2132568359375
	20th percentile R2(t,t-1): 0.7658944725990295
	taking step 3
	median kurtosis: 23.83907127380371
	20th percentile R2(t,t-1): 0.9089255332946777
	taking step 4
	median kurtosis: 16.483577728271484
	20th percentile R2(t,t-1): 0.9428997039794922
	taking step 5
	median kurtosis: 12.018216133117676
	20th percentile R2(t,t-1): 0.9613077640533447
	taking step 6
	median kurtosis: 9.285207748413086
	20th percentile R2(t,t-1): 0.9742846488952637
stopping after 6 steps
covariate-adjusted NAM not found; computing and saving


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing SVD
performing association test
computing neighborhood-level FDRs
3
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.0016998300169983002 , 3 PCs used
total r^2 between top 3 NAM PCs and outcome is 0.13


In [88]:
sc_uni['Prednisone_use'] = res.p

In [44]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Pred_use, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      covs = d.samplem[['First_biop', 'Final_Activity', 
                                                         'Responder_Status', 
                                                        'Final_ISN_[III][V]', 'Final_ISN_[IV]',
                                                        'Final_ISN_[IV][V]', 'Final_ISN_[V]',
                                                        'Final_Site_Einstein', 'Final_Site_JHU', 
                                                        'Final_Site_MUSC', 'Final_Site_NYU',
                                                        'Final_Site_Rochester', 'Final_Site_Texas Tech', 
                                                        'Final_Site_UCSD', 'Final_Site_UCSF']], 
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 53.41489028930664
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 35.2132568359375
	20th percentile R2(t,t-1): 0.7658944725990295
	taking step 3
	median kurtosis: 23.83907127380371
	20th percentile R2(t,t-1): 0.9089255332946777
	taking step 4
	median kurtosis: 16.483577728271484
	20th percentile R2(t,t-1): 0.9428997039794922
	taking step 5
	median kurtosis: 12.018216133117676
	20th percentile R2(t,t-1): 0.9613077640533447
	taking step 6
	median kurtosis: 9.285207748413086
	20th percentile R2(t,t-1): 0.9742846488952637
stopping after 6 steps
covariate-adjusted NAM not found; computing and saving


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing SVD
performing association test
computing neighborhood-level FDRs
3
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.011198880111988802 , 3 PCs used
total r^2 between top 3 NAM PCs and outcome is 0.16


In [46]:
sc_cond['Prednisone_use'] = res.p

In [10]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Pred_use, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      covs = d.samplem[['First_biop', 'Final_Activity', 
                                                         'Responder_Status', 'Final_Chronicity',
                                                        'Final_ISN_[III][V]', 'Final_ISN_[IV]',
                                                        'Final_ISN_[IV][V]', 'Final_ISN_[V]',
                                                        'Final_Site_Einstein', 'Final_Site_JHU', 
                                                        'Final_Site_MUSC', 'Final_Site_NYU',
                                                        'Final_Site_Rochester', 'Final_Site_Texas Tech', 
                                                        'Final_Site_UCSD', 'Final_Site_UCSF']], 
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 53.41489028930664
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 35.2132568359375
	20th percentile R2(t,t-1): 0.7658944725990295
	taking step 3
	median kurtosis: 23.83907127380371
	20th percentile R2(t,t-1): 0.9089255332946777
	taking step 4
	median kurtosis: 16.483577728271484
	20th percentile R2(t,t-1): 0.9428997039794922
	taking step 5
	median kurtosis: 12.018216133117676
	20th percentile R2(t,t-1): 0.9613077640533447
	taking step 6
	median kurtosis: 9.285207748413086
	20th percentile R2(t,t-1): 0.9742846488952637
stopping after 6 steps
covariate-adjusted NAM not found; computing and saving


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing SVD
performing association test
computing neighborhood-level FDRs
3
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.008599140085991401 , 3 PCs used
total r^2 between top 3 NAM PCs and outcome is 0.17


In [12]:
sc_chron['Prednisone_use'] = res.p

## Single Nuclear

In [56]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/first_biop_pred_use_site/sn_meta_preduse.csv')
print(meta.shape)

(36541, 33)


In [58]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/first_biop_pred_use_site/sn_harmony_preduse.csv')
print(harmony.shape)

(36541, 20)


In [60]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/first_biop_pred_use_site/sn_umap_preduse.csv')
print(umap.shape)

(36541, 2)


In [67]:
# Start by making anndata object
d = mad.MultiAnnData(X=harmony, obs=meta, sampleid="sample")
#d.obs_to_sample(['Type', 'Site',
#                 'Responder_Status', 'Sex',
#                 'Age', 'Race',
#                'Ethnicity', 'ISN',
#               'Activity', "Chronicity",
#                 'processing_batch'])
d.obs_to_sample(['Sex', 'Age', 'Final_Chronicity', 'Final_Activity', 'First_biop', 'Pred_use', 'Responder_Status', 'Race_[A]',
        'Race_[B]', 'Race_[U]',
       'Race_[W]', 'Final_ISN_[III]', 'Final_ISN_[III][V]', 'Final_ISN_[IV]',
       'Final_ISN_[IV][V]', 'Final_ISN_[V]', 'Final_Site_Cincinnati',
       'Final_Site_JHU', 'Final_Site_Michigan',
       'Final_Site_MUSC', 'Final_Site_Northwell', 'Final_Site_NYU',
       'Final_Site_Rochester', 'Final_Site_Texas Tech', 
       'Final_Site_UCSF'])
d.samplem.head()

['cell' 'sample' 'cluster_names' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


,Sex,Age,Final_Chronicity,Final_Activity,First_biop,Pred_use,Responder_Status,Race_[A],Race_[B],Race_[U],...,Final_ISN_[V],Final_Site_Cincinnati,Final_Site_JHU,Final_Site_Michigan,Final_Site_MUSC,Final_Site_Northwell,Final_Site_NYU,Final_Site_Rochester,Final_Site_Texas Tech,Final_Site_UCSF
sample,,,,,,,,,,,,,,,,,,,,,
AMPSLEkid_cells_0147,1.0,0.788549,9.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
AMPSLEkid_cells_0366,2.0,-1.232745,7.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
AMPSLEkid_cells_0849,1.0,0.237287,9.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0880,1.0,0.880426,NaN,NaN,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0883,2.0,-1.416499,NaN,NaN,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [68]:
umap.index = d.obs.index
d.obsm['X_umap'] = umap

In [69]:
np.random.seed(0) 
cna.pp.knn(d)

computing default knn graph


In [70]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Pred_use, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 12.009225845336914
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 8.349165916442871
	20th percentile R2(t,t-1): 0.7731621861457825
	taking step 3
	median kurtosis: 5.703102111816406
	20th percentile R2(t,t-1): 0.9103659629821778
stopping after 3 steps
covariate-adjusted NAM not found; computing and saving
computing SVD
performing association test


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_association.py:57: UserWarning: data supported use of 20 NAM PCs, which is the maximum considered. Consider allowing more PCs by using the "ks" argument.
  warnings.warn(('data supported use of {} NAM PCs, which is the maximum considered. '+\


computing neighborhood-level FDRs
20
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.40335966403359663 , 20 PCs used
total r^2 between top 20 NAM PCs and outcome is 0.57


# Chronicity

## Single Cell

In [5]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/chronicity/sc_meta.csv')
print(meta.shape)

(108462, 39)


In [6]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/chronicity/sc_harmony.csv')
print(harmony.shape)

(108462, 20)


In [7]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/chronicity/sc_umap.csv')
print(umap.shape)

(108462, 2)


In [9]:
res = cna_test(meta,harmony, umap, 'Final_Chronicity', covars = ['First_biop', 'Responder_Status'])

(108462, 39)
(108462, 20)
(108462, 2)
['cell' 'sample' 'final_annotation' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies
computing default knn graph


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 49.563255310058594
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 32.62270164489746
	20th percentile R2(t,t-1): 0.7670874118804931
	taking step 3
	median kurtosis: 22.344921112060547
	20th percentile R2(t,t-1): 0.9100062370300293
	taking step 4
	median kurtosis: 15.70623779296875
	20th percentile R2(t,t-1): 0.9437666416168213
	taking step 5
	median kurtosis: 11.611591339111328
	20th percentile R2(t,t-1): 0.9620700478553772
	taking step 6
	median kurtosis: 9.129266262054443
	20th percentile R2(t,t-1): 0.9746026992797852
stopping after 6 steps
covariate-adjusted NAM not found; computing and saving


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing SVD
performing association test


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_association.py:74: UserWarning: global association p-value attained minimal possible value. Consider increasing Nnull
  warnings.warn('global association p-value attained minimal possible value. '+\


computing neighborhood-level FDRs
8
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 9.999000099990002e-05 , 8 PCs used
total r^2 between top 8 NAM PCs and outcome is 0.53


In [10]:
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/chronicity/sc_conditional_ncorr.csv", 
               res.ncorrs, delimiter=",")
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/chronicity/sc_conditional_fdrs.csv", 
               res.fdrs, delimiter=",")

## Single Nuclear

In [11]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/chronicity/sn_meta.csv')
print(meta.shape)

(30363, 32)


In [12]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/chronicity/sn_harmony.csv')
print(harmony.shape)

(30363, 20)


In [13]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/chronicity/sn_umap.csv')
print(umap.shape)

(30363, 2)


In [14]:
res = cna_test(meta,harmony, umap, 'Final_Chronicity', covars = ['First_biop', 'Responder_Status'])

(30363, 32)
(30363, 20)
(30363, 2)
['cell' 'sample' 'final_annotation' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies
computing default knn graph


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 10.494342803955078
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 7.402501106262207
	20th percentile R2(t,t-1): 0.7776154518127442
	taking step 3
	median kurtosis: 5.242842674255371
	20th percentile R2(t,t-1): 0.9123160123825074
stopping after 3 steps
covariate-adjusted NAM not found; computing and saving
computing SVD
performing association test


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing neighborhood-level FDRs
16
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.038696130386961305 , 16 PCs used
total r^2 between top 16 NAM PCs and outcome is 0.78


In [15]:
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/chronicity/sn_conditional_ncorr.csv", 
               res.ncorrs, delimiter=",")
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/chronicity/sn_conditional_fdrs.csv", 
               res.fdrs, delimiter=",")

# Activity

## Single Cell

In [6]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/activity/sc_meta.csv')
print(meta.shape)

(108462, 39)


In [7]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/activity/sc_harmony.csv')
print(harmony.shape)

(108462, 20)


In [8]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/activity/sc_umap.csv')
print(umap.shape)

(108462, 2)


In [10]:
res = cna_test(meta, harmony, umap, 'Final_Activity')

(108462, 39)
(108462, 20)
(108462, 2)
['cell' 'sample' 'cluster_names' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies
computing default knn graph


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 49.563255310058594
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 32.62270164489746
	20th percentile R2(t,t-1): 0.7670874118804931
	taking step 3
	median kurtosis: 22.344921112060547
	20th percentile R2(t,t-1): 0.9100062370300293
	taking step 4
	median kurtosis: 15.70623779296875
	20th percentile R2(t,t-1): 0.9437666416168213
	taking step 5
	median kurtosis: 11.611591339111328
	20th percentile R2(t,t-1): 0.9620700478553772
	taking step 6
	median kurtosis: 9.129266262054443
	20th percentile R2(t,t-1): 0.9746026992797852
stopping after 6 steps
covariate-adjusted NAM not found; computing and saving


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing SVD
performing association test
computing neighborhood-level FDRs
9
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.0004999500049995 , 9 PCs used
total r^2 between top 9 NAM PCs and outcome is 0.24


In [9]:
res = cna_test(meta, harmony, umap, 'Final_Activity', 
               covars = ['Pred_use', 'Final_ISN_[IV]',
                         'Final_ISN_[IV][V]',
                         'Final_ISN_[V]',
                         'Final_Site_JHU',
                         'Final_Site_NYU'])

(108462, 39)
(108462, 20)
(108462, 2)
['cell' 'sample' 'cluster_names' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


computing default knn graph


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 49.563255310058594
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 32.62270164489746
	20th percentile R2(t,t-1): 0.7670874118804931
	taking step 3
	median kurtosis: 22.344921112060547
	20th percentile R2(t,t-1): 0.9100062370300293
	taking step 4
	median kurtosis: 15.70623779296875
	20th percentile R2(t,t-1): 0.9437666416168213
	taking step 5
	median kurtosis: 11.611591339111328
	20th percentile R2(t,t-1): 0.9620700478553772
	taking step 6
	median kurtosis: 9.129266262054443
	20th percentile R2(t,t-1): 0.9746026992797852
stopping after 6 steps
covariate-adjusted NAM not found; computing and saving


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing SVD
performing association test
computing neighborhood-level FDRs
9
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.0927907209279072 , 9 PCs used
total r^2 between top 9 NAM PCs and outcome is 0.14


## Single Nuclear

In [91]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/activity/sn_meta.csv')
print(meta.shape)

(30363, 11)


In [92]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/activity/sn_harmony.csv')
print(harmony.shape)

(30363, 20)


In [93]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/activity/sn_umap.csv')
print(umap.shape)

(30363, 2)


In [94]:
# Start by making anndata object
d = mad.MultiAnnData(X=harmony, obs=meta, sampleid="sample")
#d.obs_to_sample(['Type', 'Site',
#                 'Responder_Status', 'Sex',
#                 'Age', 'Race',
#                'Ethnicity', 'ISN',
#               'Activity', "Chronicity",
#                 'processing_batch'])
d.obs_to_sample(['Final_Chronicity', 'Final_Activity'])
d.samplem.head()

['cell' 'Sex' 'sample' 'cluster_names' 'Responder.Status' 'Race'
 'Final_ISN' 'Type']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


,Final_Chronicity,Final_Activity
sample,,
AMPSLEkid_cells_0147,9.0,1.0
AMPSLEkid_cells_0366,7.0,2.0
AMPSLEkid_cells_0849,9.0,1.0
AMPSLEkid_cells_1131,2.0,10.0
AMPSLEkid_cells_1135,3.0,7.0


In [95]:
umap.index = d.obs.index
d.obsm['X_umap'] = umap

In [96]:
np.random.seed(0) 
cna.pp.knn(d)

computing default knn graph


In [97]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Activity, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 10.494342803955078
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 7.402501106262207
	20th percentile R2(t,t-1): 0.7776154518127442
	taking step 3
	median kurtosis: 5.242842674255371
	20th percentile R2(t,t-1): 0.9123160123825074
stopping after 3 steps
covariate-adjusted NAM not found; computing and saving
computing SVD
performing association test


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing neighborhood-level FDRs
11
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.014598540145985401 , 11 PCs used
total r^2 between top 11 NAM PCs and outcome is 0.70


In [99]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Activity, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], # I asked the method to consider up to 10 PCs because
                                      covs = d.samplem[['Final_Chronicity']]
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

covariate-adjusted NAM not found; computing and saving
computing SVD
performing association test


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing neighborhood-level FDRs
15
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.024897510248975102 , 15 PCs used
total r^2 between top 15 NAM PCs and outcome is 0.76


In [100]:
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/activity/sn_chronicity_cond_ncorr.csv", 
               res.ncorrs, delimiter=",")
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/activity/sn_chronicity_cond_fdrs.csv", 
               res.fdrs, delimiter=",")

# ISN

## Single Cell

In [11]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/ISN/sc_meta.csv')
print(meta.shape)

(116401, 39)


In [12]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/ISN/sc_harmony.csv')
print(harmony.shape)

(116401, 20)


In [13]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/ISN/sc_umap.csv')
print(umap.shape)

(116401, 2)


In [15]:
res = cna_test(meta, harmony, umap, 'Final_ISN_[IV]')

(116401, 39)
(116401, 20)
(116401, 2)
['cell' 'sample' 'final_annotation' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies
computing default knn graph


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 53.41489028930664
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 35.2132568359375
	20th percentile R2(t,t-1): 0.7658944725990295
	taking step 3
	median kurtosis: 23.83907127380371
	20th percentile R2(t,t-1): 0.9089255332946777
	taking step 4
	median kurtosis: 16.483577728271484
	20th percentile R2(t,t-1): 0.9428997039794922
	taking step 5
	median kurtosis: 12.018216133117676
	20th percentile R2(t,t-1): 0.9613077640533447
	taking step 6
	median kurtosis: 9.285207748413086
	20th percentile R2(t,t-1): 0.9742846488952637
stopping after 6 steps
covariate-adjusted NAM not found; computing and saving


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing SVD
performing association test
computing neighborhood-level FDRs
13
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.0163983601639836 , 13 PCs used
total r^2 between top 13 NAM PCs and outcome is 0.20


In [17]:
res = cna_test(meta, harmony, umap, 'Final_ISN_[IV]', covars = ['Final_Activity'])

(116401, 39)
(116401, 20)
(116401, 2)
['cell' 'sample' 'final_annotation' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies
computing default knn graph


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 53.41489028930664
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 35.2132568359375
	20th percentile R2(t,t-1): 0.7658944725990295
	taking step 3
	median kurtosis: 23.83907127380371
	20th percentile R2(t,t-1): 0.9089255332946777
	taking step 4
	median kurtosis: 16.483577728271484
	20th percentile R2(t,t-1): 0.9428997039794922
	taking step 5
	median kurtosis: 12.018216133117676
	20th percentile R2(t,t-1): 0.9613077640533447
	taking step 6
	median kurtosis: 9.285207748413086
	20th percentile R2(t,t-1): 0.9742846488952637
stopping after 6 steps
covariate-adjusted NAM not found; computing and saving


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing SVD
performing association test
computing neighborhood-level FDRs
1
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.24677532246775322 , 1 PCs used
total r^2 between top 1 NAM PCs and outcome is 0.03


In [16]:
res = cna_test(meta, harmony, umap, 'Final_ISN_[V]')

(116401, 39)
(116401, 20)
(116401, 2)
['cell' 'sample' 'final_annotation' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies
computing default knn graph


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 53.41489028930664
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 35.2132568359375
	20th percentile R2(t,t-1): 0.7658944725990295
	taking step 3
	median kurtosis: 23.83907127380371
	20th percentile R2(t,t-1): 0.9089255332946777
	taking step 4
	median kurtosis: 16.483577728271484
	20th percentile R2(t,t-1): 0.9428997039794922
	taking step 5
	median kurtosis: 12.018216133117676
	20th percentile R2(t,t-1): 0.9613077640533447
	taking step 6
	median kurtosis: 9.285207748413086
	20th percentile R2(t,t-1): 0.9742846488952637
stopping after 6 steps
covariate-adjusted NAM not found; computing and saving


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing SVD
performing association test
computing neighborhood-level FDRs
3
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.1351864813518648 , 3 PCs used
total r^2 between top 3 NAM PCs and outcome is 0.06


## Single Nuclear

In [117]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/ISN/sn_meta.csv')
print(meta.shape)

(36541, 27)


In [116]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/ISN/sn_harmony.csv')
print(harmony.shape)

(36541, 20)


In [118]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/ISN/sn_umap.csv')
print(umap.shape)

(36541, 2)


In [119]:
# Start by making anndata object
d = mad.MultiAnnData(X=harmony, obs=meta, sampleid="sample")
#d.obs_to_sample(['Type', 'Site',
#                 'Responder_Status', 'Sex',
#                 'Age', 'Race',
#                'Ethnicity', 'ISN',
#               'Activity', "Chronicity",
#                 'processing_batch'])
d.obs_to_sample(['Final_ISN_[III]', 'Final_ISN_[III][V]', 'Final_ISN_[IV]',
                 'Final_ISN_[IV][V]', 'Final_ISN_[V]'])
d.samplem.head()

['cell' 'Sex' 'sample' 'cluster_names' 'Responder.Status' 'Race'
 'Final_ISN' 'Type' 'cell.1' 'Sex.1' 'sample.1' 'cluster_names.1'
 'Responder.Status.1' 'Race.1' 'Final_ISN.1' 'Type.1']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


,Final_ISN_[III],Final_ISN_[III][V],Final_ISN_[IV],Final_ISN_[IV][V],Final_ISN_[V]
sample,,,,,
AMPSLEkid_cells_0147,1.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0366,0.0,1.0,0.0,0.0,0.0
AMPSLEkid_cells_0849,1.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0880,0.0,0.0,0.0,1.0,0.0
AMPSLEkid_cells_0883,0.0,0.0,0.0,1.0,0.0


In [ ]:
umap.index = d.obs.index
d.obsm['X_umap'] = umap

In [122]:
np.random.seed(0) 
cna.pp.knn(d)

computing default knn graph


In [123]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem['Final_ISN_[IV]'], #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 12.009225845336914
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 8.349165916442871
	20th percentile R2(t,t-1): 0.7731621861457825
	taking step 3
	median kurtosis: 5.703102111816406
	20th percentile R2(t,t-1): 0.9103659629821778
stopping after 3 steps
covariate-adjusted NAM not found; computing and saving
computing SVD
performing association test


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing neighborhood-level FDRs
14
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.7933206679332067 , 14 PCs used
total r^2 between top 14 NAM PCs and outcome is 0.36


In [124]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem['Final_ISN_[IV][V]'], #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
8
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.6141385861413858 , 8 PCs used
total r^2 between top 8 NAM PCs and outcome is 0.30


In [116]:
pd.DataFrame.from_dict(sc_uni, orient='index').T.to_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/univariate_cna.csv')

In [87]:
sc_cond['Sex'] = "NA"
sc_cond['Race'] = "NA"
sc_cond['Age'] = "NA"
pd.DataFrame.from_dict(sc_cond, orient='index').T.to_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/conditional_cna.csv')

In [18]:
sc_chron['Sex'] = "NA"
sc_chron['Race'] = "NA"
sc_chron['Age'] = "NA"
sc_chron['Final_Chronicity'] = "NA"
sc_chron['Final_Activity'] = "NA"
sc_chron['Final_ISN'] = "NA"
sc_chron['Race'] = "NA"
sc_chron['Site'] = "NA"
sc_chron['First_biopsy'] = "NA"
sc_chron['Responder_Status'] = "NA"
pd.DataFrame.from_dict(sc_chron, orient='index').T.to_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/conditional_chronicity_cna.csv')

# INJURED PT

## Single Cell

In [14]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/injured_pt/sc_meta.csv')
print(meta.shape)

(116382, 40)


In [15]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/injured_pt/sc_harmony.csv')
print(harmony.shape)

(116382, 20)


In [16]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/injured_pt/sc_umap.csv')
print(umap.shape)

(116382, 2)


In [6]:
res = cna_test(meta,harmony, umap, 'injured_pt_prop')

(116382, 40)
(116382, 20)
(116382, 2)


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


['cell' 'sample' 'final_annotation' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies
computing default knn graph


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 52.936561584472656
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 34.72443389892578
	20th percentile R2(t,t-1): 0.7662303686141968
	taking step 3
	median kurtosis: 23.369674682617188
	20th percentile R2(t,t-1): 0.9089954972267151
	taking step 4
	median kurtosis: 15.963044166564941
	20th percentile R2(t,t-1): 0.94291832447052
	taking step 5
	median kurtosis: 11.406298637390137
	20th percentile R2(t,t-1): 0.9615656614303589
	taking step 6
	median kurtosis: 8.651910781860352
	20th percentile R2(t,t-1): 0.9743394613265991
stopping after 6 steps
covariate-adjusted NAM not found; computing and saving


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing SVD
performing association test


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_association.py:74: UserWarning: global association p-value attained minimal possible value. Consider increasing Nnull
  warnings.warn('global association p-value attained minimal possible value. '+\


computing neighborhood-level FDRs
6
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 9.999000099990002e-05 , 6 PCs used
total r^2 between top 6 NAM PCs and outcome is 0.45


In [7]:
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/injured_pt/sc_ncorr.csv", 
               res.ncorrs, delimiter=",")
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/injured_pt/sc_fdrs.csv", 
               res.fdrs, delimiter=",")

In [17]:
cna_object, res = cna_test(meta,harmony, umap, 'injured_pt_prop', 
               covars = ['First_biop',
                         'Age',
                         'Final_Site_JHU',
                         'Final_Site_NYU',
                         'Responder_Status', 
                         'Final_Chronicity'],
                          ret_cna_obj = True)

(116382, 40)
(116382, 20)
(116382, 2)
['cell' 'sample' 'final_annotation' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies
computing default knn graph


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 52.936561584472656
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 34.72443389892578
	20th percentile R2(t,t-1): 0.7662303686141968
	taking step 3
	median kurtosis: 23.369674682617188
	20th percentile R2(t,t-1): 0.9089954972267151
	taking step 4
	median kurtosis: 15.963044166564941
	20th percentile R2(t,t-1): 0.94291832447052
	taking step 5
	median kurtosis: 11.406298637390137
	20th percentile R2(t,t-1): 0.9615656614303589
	taking step 6
	median kurtosis: 8.651910781860352
	20th percentile R2(t,t-1): 0.9743394613265991
stopping after 6 steps
covariate-adjusted NAM not found; computing and saving


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing SVD
performing association test
computing neighborhood-level FDRs
8
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.00019998000199980003 , 8 PCs used
total r^2 between top 8 NAM PCs and outcome is 0.32


In [18]:
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/injured_pt/sc_cond_ncorr.csv", 
               res.ncorrs, delimiter=",")
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/injured_pt/sc_cond_fdrs.csv", 
               res.fdrs, delimiter=",")

In [19]:
y_resid_hat = pd.DataFrame([pd.DataFrame(cna_object.uns['NAM_sampleXpc']).index.tolist(), res.yresid_hat],
             index = ['Unified_Visit', 'Predicted Chronicity']).T
y_resid_hat.to_csv("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/injured_pt/cond_y_resid_hat.csv",
                   index = False)

## Single Nuclear

In [23]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/injured_pt/sn_meta.csv')
print(meta.shape)

(36541, 34)


In [21]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/injured_pt/sn_harmony.csv')
print(harmony.shape)

(36541, 20)


In [22]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/injured_pt/sn_umap.csv')
print(umap.shape)

(36541, 2)


In [6]:
res = cna_test(meta,harmony, umap, 'injured_pt_prop')

(36541, 34)
(36541, 20)
(36541, 2)
['cell' 'sample' 'final_annotation' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies
computing default knn graph


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 12.009225845336914
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 8.349165916442871
	20th percentile R2(t,t-1): 0.7731621861457825
	taking step 3
	median kurtosis: 5.703102111816406
	20th percentile R2(t,t-1): 0.9103659629821778
stopping after 3 steps
covariate-adjusted NAM not found; computing and saving
computing SVD
performing association test


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_association.py:74: UserWarning: global association p-value attained minimal possible value. Consider increasing Nnull
  warnings.warn('global association p-value attained minimal possible value. '+\


computing neighborhood-level FDRs
19
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 9.999000099990002e-05 , 19 PCs used
total r^2 between top 19 NAM PCs and outcome is 0.94


In [7]:
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/injured_pt/sn_ncorr.csv", 
               res.ncorrs, delimiter=",")
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/injured_pt/sn_fdrs.csv", 
               res.fdrs, delimiter=",")

In [24]:
cna_obj, res = cna_test(meta,harmony, umap, 'injured_pt_prop', 
               covars = ['First_biop',
                         'Age',
                         'Final_Site_JHU',
                         'Final_Site_NYU',
                         'Responder_Status',
                         'Final_Chronicity'],
                        ret_cna_obj = True)

(36541, 34)
(36541, 20)
(36541, 2)
['cell' 'sample' 'final_annotation' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies
computing default knn graph


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 12.009225845336914
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 8.349165916442871
	20th percentile R2(t,t-1): 0.7731621861457825
	taking step 3
	median kurtosis: 5.703102111816406
	20th percentile R2(t,t-1): 0.9103659629821778
stopping after 3 steps
covariate-adjusted NAM not found; computing and saving
computing SVD
performing association test


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_association.py:74: UserWarning: global association p-value attained minimal possible value. Consider increasing Nnull
  warnings.warn('global association p-value attained minimal possible value. '+\


computing neighborhood-level FDRs
15
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 9.999000099990002e-05 , 15 PCs used
total r^2 between top 15 NAM PCs and outcome is 0.96


In [25]:
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/injured_pt/sn_cond_ncorr.csv", 
               res.ncorrs, delimiter=",")
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/LOH/injured_pt/sn_cond_fdrs.csv", 
               res.fdrs, delimiter=",")